In [1]:
import tensorflow as tf
import numpy as np
import math
import os
import random
import numpy as np
import pandas as pd
from pandas import DataFrame
import pickle as pickle
import math
import copy

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
with open('C:\\Users\\sara1\\Desktop\\Workingdata\\state_features.txt') as f:
    state_features = f.read().split()
print (state_features)
print (len(state_features))

['sofa_score', 'sirs', 'age', 'weight', 'gender', 'gcs', 'sofa_24hours', 'bilirubin_total', 'bicarbonate', 'hemoglobin', 'chloride', 'calcium', 'potassium', 'sodium', 'lactate', 'glucose', 'respiratory_rate_total', 'fio2', 'platelet', 'wbc', 'albumin', 'creatinine', 'heart_rate', 'temperature', 'spo2', 'vaso_rate', 'vaso_amount', 'urineoutput']
28


In [4]:
df = pd.read_csv('C:\\Users\\sara1\\Desktop\\Workingdata\\rl_train_below65.csv')

In [5]:
df.head()

,stay_id,sofa_score,sirs,age,weight,gender,gcs,sofa_24hours,bilirubin_total,bicarbonate,...,creatinine,heart_rate,temperature,spo2,vaso_rate,vaso_amount,urineoutput,died_in_hosp,vaso_input,reward
0,38563000,0.0,1.0,0.425532,0.107514,1.0,1.0,0.043478,0.065682,0.8125,...,0.105287,0.506438,0.563117,0.959184,0.000483,0.039987,0.057143,0,2.0,-0.25
1,38563000,0.0,1.0,0.425532,0.107514,1.0,1.0,0.086957,0.065682,0.3750,...,0.105287,0.506438,0.563117,0.959184,0.000483,0.039987,0.057143,0,2.0,0.00
2,38563000,0.0,1.0,0.425532,0.107514,1.0,1.0,0.173913,0.065682,0.0000,...,0.105287,0.506438,0.563117,0.959184,0.000483,0.039987,0.057143,0,2.0,-0.25
3,38563000,0.0,1.0,0.425532,0.107514,1.0,1.0,0.043478,0.065682,0.3125,...,0.105287,0.506438,0.563117,0.959184,0.000483,0.039987,0.057143,0,2.0,-0.25
4,38563000,0.0,1.0,0.425532,0.107514,1.0,1.0,0.086957,0.065682,0.3750,...,0.105287,0.506438,0.563117,0.959184,0.000483,0.039987,0.057143,0,2.0,-0.25


In [6]:
val_df = pd.read_csv('C:\\Users\\sara1\\Desktop\\Workingdata\\rl_val_below65.csv')

In [7]:
test_df = pd.read_csv('C:\\Users\\sara1\\Desktop\\Workingdata\\rl_test_below65.csv')

In [8]:
REWARD_THRESHOLD = 20
reg_lambda = 5

In [9]:
# PER important weights and params
per_flag = True
beta_start = 0.9
df['prob'] = abs(df['reward'])
temp = 1.0/df['prob']
#temp[temp == float('Inf')] = 1.0
df['imp_weight'] = pow((1.0/len(df) * temp), beta_start)

In [10]:
hidden_1_size = 128
hidden_2_size = 128
#  Q-network uses Leaky ReLU activation
class Qnetwork():
    def __init__(self):
        self.phase = tf.compat.v1.placeholder(tf.bool)

        self.num_actions = 5

        self.input_size = len(state_features)

        self.state = tf.compat.v1.placeholder(tf.float32, shape=[None, self.input_size],name="input_state")

        self.fc_1 = tf.keras.layers.Dense(hidden_1_size, activation=None)(self.state)
        self.fc_1_bn = tf.keras.layers.BatchNormalization(center=True, scale=True)(self.fc_1, training=self.phase)
        self.fc_1_ac = tf.keras.activations.relu(self.fc_1_bn, alpha=0.01)
        self.fc_2 = tf.keras.layers.Dense(hidden_2_size, activation=None)(self.fc_1_ac)
        self.fc_2_bn = tf.keras.layers.BatchNormalization(center=True, scale=True)(self.fc_2, training=self.phase)
        self.fc_2_ac = tf.keras.activations.relu(self.fc_2_bn, alpha=0.01)

        # advantage and value streams
        self.streamA,self.streamV = tf.split(self.fc_2_ac,2,axis=1)
        self.AW = tf.Variable(tf.random.normal([hidden_2_size//2,self.num_actions]))
        self.VW = tf.Variable(tf.random.normal([hidden_2_size//2,1]))
        self.Advantage = tf.matmul(self.streamA,self.AW)
        self.Value = tf.matmul(self.streamV,self.VW)

        #Then combine them together to get our final Q-values.
        self.q_output = self.Value + tf.subtract(self.Advantage,tf.reduce_mean(self.Advantage,axis=1,keepdims=True))

        #Below we obtain the loss by taking the sum of squares difference between the target and predicted Q values.
        self.targetQ = tf.compat.v1.placeholder(shape=[None],dtype=tf.float32)
        self.actions = tf.compat.v1.placeholder(shape=[None],dtype=tf.int32)
        self.actions_onehot = tf.one_hot(self.actions,self.num_actions,dtype=tf.float32)

        # Importance sampling weights for PER, used in network update
        self.imp_weights = tf.compat.v1.placeholder(shape=[None], dtype=tf.float32)

        # select the Q values for the actions that would be selected
        self.Q = tf.reduce_sum(tf.multiply(self.q_output, self.actions_onehot), axis=1) # batch size x 1 vector

        # regularisation penalises the network when it produces rewards that are above the
        # reward threshold, to ensure reasonable Q-value predictions
        self.reg_vector = tf.maximum(tf.abs(self.Q)-REWARD_THRESHOLD,0)
        self.reg_term = tf.reduce_sum(self.reg_vector)

        self.abs_error = tf.abs(self.targetQ - self.Q)

        self.td_error = tf.square(self.targetQ - self.Q)

        # below is the loss when we are not using PER
        self.old_loss = tf.reduce_mean(self.td_error)

        # as in the paper, to get PER loss we weight the squared error by the importance weights
        self.per_error = tf.multiply(self.td_error, self.imp_weights)

        # total loss is a sum of PER loss and the regularisation term
        if per_flag:
            self.loss = tf.reduce_mean(self.per_error) + reg_lambda*self.reg_term
        else:
            self.loss = self.old_loss + reg_lambda*self.reg_term

        self.trainer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.0001)
        self.update_ops = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(self.update_ops):
        # Ensures that we execute the update_ops before performing the model update, so batchnorm works
            self.update_model = self.trainer.minimize(self.loss)

In [11]:
# function is needed to update parameters between main and target network
# tf_vars are the trainable variables to update, and tau is the rate at which to update
# returns tf ops corresponding to the updates
def update_target_graph(tf_vars,tau):
    total_vars = len(tf_vars)
    op_holder = []
    for idx,var in enumerate(tf_vars[0:int(total_vars/2)]):
        op_holder.append(tf_vars[idx+int(total_vars/2)].assign((var.value()*tau) + ((1-tau)*tf_vars[idx+int(total_vars/2)].value())))
    return op_holder

In [12]:
def update_target(op_holder,sess):
    for op in op_holder:
        sess.run(op)

In [13]:
# define an action mapping - how to get an id representing the action from the (vaso) tuple
action_map = {}
count = 0
for vaso in range(5):
    action_map[(vaso)] = count
    count += 1

In [14]:
# generates batches for the Q network - depending on train and eval_type, can select data from train/val/test sets.
def process_train_batch(size):
    if per_flag:
        # uses prioritised exp replay
        a = df.sample(n=size, weights=df['prob'])
    else:
        a = df.sample(n=size)
    states = None
    actions = None
    rewards = None
    next_states = None
    next_actions = None
    done_flags = None
    for i in a.index:
        cur_state = a.loc[i,state_features]
        vaso = int(a.loc[i, 'vaso_input'])
        action = action_map[vaso]
        reward = a.loc[i,'reward']

        if i != df.index[-1]:
            # if not terminal step in trajectory
            if df.loc[i, 'stay_id'] == df.loc[i+1, 'stay_id']:
                next_state = df.loc[i + 1, state_features]
                next_vaso = int(df.loc[i+1, 'vaso_input'])
                next_action = action_map[vaso]
                done = 0
            else:
                # trajectory is finished
                next_state = np.zeros(len(cur_state))
                next_action = 0
                done = 1
        else:
            # last entry in df is the final state of that trajectory
            next_state = np.zeros(len(cur_state))
            next_action = 0
            done = 1

        if states is None:
            states = copy.deepcopy(cur_state)
        else:
            states = np.vstack((states,cur_state))

        if actions is None:
            actions = [action]
        else:
            actions = np.vstack((actions,action))

        if rewards is None:
            rewards = [reward]
        else:
            rewards = np.vstack((rewards,reward))

        if next_states is None:
            next_states = copy.deepcopy(next_state)
        else:
            next_states = np.vstack((next_states,next_state))

        if next_actions is None:
            next_actions = [next_action]
        else:
            next_actions = np.vstack((next_actions,next_action))

        if done_flags is None:
            done_flags = [done]
        else:
            done_flags = np.vstack((done_flags,done))

    return (states, np.squeeze(actions), np.squeeze(rewards), next_states,np.squeeze(next_actions), np.squeeze(done_flags), a)


In [15]:
# extract chunks of length size from the relevant dataframe, and yield these to the caller
def process_eval_batch(size, eval_type = None):
    if eval_type is None:
        raise Exception('Provide eval_type to process_eval_batch')
    elif eval_type == 'train':
        a = df.copy()
    elif eval_type == 'val':
        a = val_df.copy()
    elif eval_type == 'test':
        a = test_df.copy()
    else:
        raise Exception('Unknown eval_type')

    count = 0
    while count < len(a.index):
        states = None
        actions = None
        rewards = None
        next_states = None
        next_actions = None
        done_flags = None

        start_idx = count
        end_idx = min(len(a.index), count+size)
        segment = a.index[start_idx:end_idx]

        for i in segment:
            cur_state = a.loc[i,state_features]
            vaso = int(a.loc[i, 'vaso_input'])
            action = action_map[vaso]
            reward = a.loc[i,'reward']

            if i != a.index[-1]:
                # if not terminal step in trajectory
                if a.loc[i, 'stay_id'] == a.loc[i+1, 'stay_id']:
                    next_state = a.loc[i + 1, state_features]
                    next_vaso = int(a.loc[i+1, 'vaso_input'])
                    next_action = action_map[vaso]
                    done = 0
                else:
                    # trajectory is finished
                    next_state = np.zeros(len(cur_state))
                    next_action = 0
                    done = 1

            else:
                # last entry in df is the final state of that trajectory
                next_state = np.zeros(len(cur_state))
                done = 1

            if states is None:
                states = copy.deepcopy(cur_state)
            else:
                states = np.vstack((states,cur_state))

            if actions is None:
                actions = [action]
            else:
                actions = np.vstack((actions,action))

            if rewards is None:
                rewards = [reward]
            else:
                rewards = np.vstack((rewards,reward))

            if next_states is None:
                next_states = copy.deepcopy(next_state)
            else:
                next_states = np.vstack((next_states,next_state))

            if next_actions is None:
                next_actions = [next_action]
            else:
                next_actions = np.vstack((next_actions,next_action))

            if done_flags is None:
                done_flags = [done]
            else:
                done_flags = np.vstack((done_flags,done))

        yield (states, np.squeeze(actions), np.squeeze(rewards), next_states,np.squeeze(next_actions), np.squeeze(done_flags), a)

        count += size


In [16]:
#  Used to run diagnostics on the train set
phys_q_train = []
phys_actions_tr = []
def train_set_performance():
    count = 0
    global phys_q_train
    global phys_actions
    phys_q_train = []
    phys_actions_tr = []
    for r in df.index:
        cur_state = [df.loc[r,state_features]]
        vaso = int(df.loc[r, 'vaso_input'])
        action = action_map[vaso]
        output_q = np.squeeze(sess.run(mainQN.q_output, feed_dict = {mainQN.state : cur_state, mainQN.phase : False}))
        phys_q_train.append(output_q[action])
        phys_actions_tr.append(action)
        count += 1

In [17]:
def do_eval(eval_type):
    gen = process_eval_batch(size = 1000, eval_type=eval_type)

    phys_q_ret = []
    actions_ret = []
    error_ret = 0

    for b in gen:
        states,actions,rewards,next_states, next_actions, done_flags, _ = b

        # Q values for the next timestep from target network, as part of the target Q calculation
        Q2 = sess.run(targetQN.q_output,feed_dict={targetQN.state:next_states, targetQN.phase : 0})

        # handles the case when a trajectory is finished
        end_multiplier = 1 - done_flags

        # Using the next actions, find the q value for the next state/action pairs
        next_state_q = Q2[range(len(Q2)),next_actions]

        # definition of target Q
        targetQ = rewards + (gamma*next_state_q * end_multiplier)

        # get the output q's, actions, and loss
        q_output, abs_error = sess.run([mainQN.q_output, mainQN.abs_error], \
            feed_dict={mainQN.state:states,
                       mainQN.targetQ:targetQ,
                       mainQN.actions:actions,
                       mainQN.phase:False})

        phys_q = q_output[range(len(q_output)), actions]
        error = np.mean(abs_error)

        #  update the return vals
        phys_q_ret.extend(phys_q)
        actions_ret.extend(actions)
        error_ret += error

    return phys_q_ret, actions_ret, error_ret

In [18]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # Don't use all GPUs
config.allow_soft_placement = True  # Enable manual control

In [19]:
def do_save_results():
    # get the chosen actions for the train, val, and test set when training is complete.
    phys_q_train, phys_actions_train, _ =  do_eval(eval_type = 'train')
    phys_q_val, phys_actions_val, _ = do_eval(eval_type = 'val')
    phys_q_test, phys_actions_test, _ = do_eval(eval_type = 'test')

    # save everything for later - they're used in policy evaluation and when generating plots
    with open(save_dir + 'phys_actions_train.p', 'wb') as f:
        pickle.dump(phys_actions_train, f)
    with open(save_dir + 'phys_actions_val.p', 'wb') as f:
        pickle.dump(phys_actions_val, f)
    with open(save_dir + 'phys_actions_test.p', 'wb') as f:
        pickle.dump(phys_actions_test, f)

    with open(save_dir + 'phys_q_train.p', 'wb') as f:
        pickle.dump(phys_q_train, f)
    with open(save_dir + 'phys_q_val.p', 'wb') as f:
        pickle.dump(phys_q_val, f)
    with open(save_dir + 'phys_q_test.p', 'wb') as f:
        pickle.dump(phys_q_test, f)
    return

In [20]:
tf.compat.v1.disable_eager_execution()

# The main training loop is here
per_alpha = 0.6 # PER hyperparameter
per_epsilon = 0.01 # PER hyperparameter
batch_size = 32
gamma = 0.99 # discount factor
num_steps = 10000 # How many steps to train for
load_model = False #Whether to load a saved model.
save_dir = "C:\\Users\\sara1\\Desktop\\Results\\SARSA_continual_below65\\"
save_path = "C:\\Users\\sara1\\Desktop\\Results\\SARSA_continual_below65\\ckpt"#The path to save our model to.
tau = 0.001 #Rate to update target network toward primary network
tf.compat.v1.reset_default_graph()
mainQN = Qnetwork()
targetQN = Qnetwork()
av_q_list = []
save_results = False

saver = tf.compat.v1.train.Saver(tf.compat.v1.global_variables())

init = tf.compat.v1.global_variables_initializer()

trainables = tf.compat.v1.trainable_variables()

target_ops = update_target_graph(trainables,tau)

#Make a path for our model to be saved in.
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

with tf.compat.v1.Session(config=config) as sess:
    if load_model == True:
        print('Trying to load model...')
        try:
            restorer = tf.compat.v1.train.import_meta_graph(save_path + '.meta')
            restorer.restore(sess, tf.train.latest_checkpoint(save_dir))
            print ("Model restored")
        except IOError:
            print ("No previous model found, running default init")
            sess.run(init)
        try:
            per_weights = pickle.load(open( save_dir + "per_weights.p", "rb" ))
            imp_weights = pickle.load(open( save_dir + "imp_weights.p", "rb" ))

            # the PER weights, governing probability of sampling, and importance sampling
            # weights for use in the gradient descent updates
            df['prob'] = per_weights
            df['imp_weight'] = imp_weights
            print ("PER and Importance weights restored")
        except IOError:
            print("No PER weights found - default being used for PER and importance sampling")
    else:
        print("Running default init")
        sess.run(init)
    print("Init done")
    net_loss = 0.0
    for i in range(num_steps):
        if save_results:
            do_save_results()
            break

        states,actions,rewards,next_states, next_actions, done_flags, sampled_df = process_train_batch(batch_size)

        # Q values for the next timestep from target network, as part of the update step
        Q2 = sess.run(targetQN.q_output,feed_dict={targetQN.state:next_states, targetQN.phase : 1})

        # handles the case when a trajectory is finished
        end_multiplier = 1 - done_flags

        # Using the next actions, find the q value for the next state/action pairs
        next_state_q = Q2[range(len(Q2)),next_actions]

        # empirical hack to make the Q values never exceed the threshold - helps learning
        next_state_q[next_state_q > REWARD_THRESHOLD] = REWARD_THRESHOLD
        next_state_q[next_state_q < -REWARD_THRESHOLD] = -REWARD_THRESHOLD


        # definition of target Q
        targetQ = rewards + (gamma*next_state_q * end_multiplier)

        # Calculate the importance sampling weights for PER
        imp_sampling_weights = np.array(sampled_df['imp_weight'] / float(max(df['imp_weight'])))
        imp_sampling_weights[np.isnan(imp_sampling_weights)] = 1
        imp_sampling_weights[imp_sampling_weights <= 0.001] = 0.001


        # Train with the batch
        _,loss, error, q_tmp = sess.run([mainQN.update_model,mainQN.loss, mainQN.abs_error, mainQN.Q], \
            feed_dict={mainQN.state:states,
                       mainQN.targetQ:targetQ,
                       mainQN.actions:actions,
                       mainQN.phase:True,
                       mainQN.imp_weights:imp_sampling_weights})

        # Update target towards main network
        update_target(target_ops,sess)

        net_loss += sum(error)

        # Set the selection weight/prob to the abs prediction error and update the importance sampling weight
        new_weights = pow((error + per_epsilon), per_alpha)
        df.loc[df.index.isin(sampled_df.index), 'prob'] = new_weights
        temp = 1.0/new_weights
        df.loc[df.index.isin(sampled_df.index), 'imp_weight'] = pow(((1.0/len(df)) * temp), beta_start)


        if i % 1000 == 0 and i > 0:
            saver.save(sess,save_path)
            print("Saved Model, step is " + str(i))

            av_loss = net_loss/(1000.0 * batch_size)
            print("Average loss is ", av_loss)
            net_loss = 0.0

            print ("Saving PER and importance weights")
            with open(save_dir + 'per_weights.p', 'wb') as f:
                pickle.dump(df['prob'], f)
            with open(save_dir + 'imp_weights.p', 'wb') as f:
                pickle.dump(df['imp_weight'], f)


        if (i % 10000==0) and i > 0:
            # run an evaluation on the validation set
            phys_q,_, mean_abs_error = do_eval(eval_type = 'val')
            print (np.mean(phys_q))
            print (mean_abs_error)
            if (i % 10000==0) and i > 0:
                print ("Saving results")
                do_save_results()
#                 break
    do_save_results()

Instructions for updating:
Colocations handled automatically by placer.
Running default init
Init done
Saved Model, step is 1000
Average loss is  5.078155596477911
Saving PER and importance weights
Saved Model, step is 2000
Average loss is  3.0810449720118194
Saving PER and importance weights
Saved Model, step is 3000
Average loss is  2.3988273158031514
Saving PER and importance weights
Saved Model, step is 4000
Average loss is  2.277058135843836
Saving PER and importance weights
Saved Model, step is 5000
Average loss is  2.3274528771177865
Saving PER and importance weights
Saved Model, step is 6000
Average loss is  2.223408730316907
Saving PER and importance weights
Saved Model, step is 7000
Average loss is  2.151845588367665
Saving PER and importance weights
Saved Model, step is 8000
Average loss is  2.022121742970776
Saving PER and importance weights
Saved Model, step is 9000
Average loss is  1.9608340614233166
Saving PER and importance weights
